In [575]:
inpf = open('17.txt')
inp = []
for i in inpf.readlines():
    inp = inp + [i.strip()]

In [576]:
horiz = []
vert = []

import re
p = re.compile("^[xy]=(\d+), [xy]=(\d+)\.\.(\d+)")
minx = 9999999
maxy = -9999999
miny = 999999999999999
maxx = -9999999
cnt = 0
for i in inp:
    clay = p.findall(i)[0]
    clay = [int(x) for x in clay]
    if i[0] == 'x':
        vert.append(clay)
        maxx = max(maxx, clay[0])
        minx = min(minx, clay[0])
        maxy = max(maxy, clay[2])
        miny = min(miny, clay[1])
    else:
        horiz.append(clay)
        maxy = max(maxy, clay[0])
        miny = min(miny, clay[0])
        maxx = max(maxx, clay[2])
        minx = min(minx, clay[1])
        
print(maxx)
print(minx)
print(maxy)
print(miny)

557
155
1806
3


In [577]:
horiz

[[360, 485, 506],
 [1610, 208, 228],
 [499, 270, 272],
 [1316, 521, 538],
 [977, 387, 389],
 [1636, 302, 329],
 [986, 227, 255],
 [350, 341, 363],
 [1494, 542, 551],
 [1548, 239, 410],
 [1062, 443, 454],
 [392, 341, 350],
 [104, 444, 462],
 [1209, 348, 369],
 [953, 504, 514],
 [1116, 312, 335],
 [929, 328, 415],
 [855, 231, 242],
 [1389, 521, 524],
 [64, 355, 382],
 [1783, 360, 363],
 [683, 431, 433],
 [102, 505, 507],
 [1291, 269, 276],
 [704, 245, 269],
 [913, 253, 264],
 [977, 167, 189],
 [397, 399, 406],
 [1347, 343, 367],
 [1780, 180, 182],
 [231, 543, 548],
 [155, 277, 287],
 [1205, 305, 307],
 [1503, 241, 261],
 [1609, 159, 179],
 [325, 296, 302],
 [634, 365, 367],
 [1596, 319, 338],
 [1533, 165, 169],
 [1155, 261, 269],
 [617, 404, 441],
 [555, 455, 461],
 [1754, 223, 231],
 [833, 377, 394],
 [87, 391, 407],
 [473, 175, 199],
 [1251, 336, 340],
 [1060, 368, 390],
 [725, 290, 297],
 [1290, 403, 408],
 [818, 274, 293],
 [544, 330, 341],
 [1116, 438, 440],
 [1256, 220, 243],
 [170

In [578]:
import numpy as np
scene = np.zeros((maxy+1,maxx+2))
scene
for v in vert:
    scene[v[1]:(v[2] + 1), v[0]] = 1
for h in horiz:
    scene[h[0],h[1]:(h[2] + 1)] = 1

#scene[0,500] = -1
scene[0:14,490:510]
start = scene.copy()


In [579]:
orig_scene = scene.copy()

In [580]:
char_map = {0:'.',1:'#', 2:'~', 3:'|',-1:'+'}

def print_scene(sc):
    height = sc.shape[0]
    width = sc.shape[1]
    for row in range(height):
        s = '{} '.format(row)
        for col in range(width):
            s += char_map[sc[row, col]]
        print (s)
        
#print_scene(scene[:,490:])
            
print_scene(scene[:,460:520])       

0 ............................................................
1 ............................................................
2 ............................................................
3 ............................................................
4 ............................................................
5 ............................................................
6 ............................................................
7 ............................................................
8 ............................................................
9 ............................................................
10 ............................................................
11 ............................#...................#...........
12 ............................#...................#...........
13 ............................#...................#...........
14 ............................#...................#...........
15 ............................#..................

1493 ...#......................#......#....####......#...........
1494 ...#......................#......#..............#...........
1495 ...#......................#......#..............#...........
1496 ...#......................#......################...........
1497 ...#......................#.................................
1498 ...#......................#.................................
1499 ...#......................#.................................
1500 ...#......................#.................................
1501 ...#......................#.................................
1502 ...#......................#.................................
1503 ...#......................#.................................
1504 ...#......................#.................................
1505 ...########################.................................
1506 ............................................................
1507 ............................................................
1508 .....

In [581]:
from collections import deque
scene = orig_scene.copy()

open_vertical = [(0,500)]

open_horizontal = []

cont = True

cnt = 0

while cont:
    cont = False
    while open_vertical:
        cont = True
        coord = open_vertical.pop()
        
        if scene[coord[0], coord[1]] == 1 or scene[coord[0]+1, coord[1]] == 3:
            continue
        
        stop = np.where(scene[coord[0]+1:,coord[1]] >= 1)[0] 

        if len(stop) == 0:
            vertical_last = maxy
        else:
            vertical_last = coord[0] + min(stop)
            if scene[vertical_last + 1, coord[1]] != 3:
                open_horizontal += [(vertical_last, coord[1])]

        scene[coord[0]+1:vertical_last + 1,coord[1]] = 3

    while open_horizontal:
        cont = True
        
        coord = open_horizontal.pop()
        horizontal = scene[coord[0]:coord[0]+2,:]

        horizontal_left = horizontal[0,0:coord[1]]
        horizontal_right = horizontal[0,coord[1]+1:]
        

        left_stop = np.where(horizontal_left == 1)[0]
        right_stop = np.where(horizontal_right == 1)[0]
        

        left_support_stop = np.where(np.isin(horizontal[1,0:coord[1]], [1,2], invert = True))[0]
        right_support_stop = np.where(np.isin(horizontal[1,coord[1] + 1:], [1,2], invert = True))[0]
 
        
        if len(left_support_stop) == 0:
            left_support = coord[1]
        else:
            left_support = max(left_support_stop) + 1
            
        if len(right_support_stop) == 0:
            right_support = coord[1]
        else:
            right_support = min(right_support_stop) + coord[1]
            
            
        right_open_ended = False
        left_open_ended = False

        if len(left_stop) == 0 or max(left_stop) < left_support:
            left_open_ended = True
            left = left_support - 1
        else:
            left = max(left_stop) + 1
        
        if len(right_stop) == 0 or min(right_stop) + coord[1] + 1 > right_support:
            right_open_ended = True
            right = right_support + 1
        else:
            right = min(right_stop) + coord[1]

        if left_open_ended or right_open_ended:
            scene[coord[0],left:right + 1] = 3
            if left_open_ended:
                open_vertical += [(coord[0], left)]
            if right_open_ended:
                open_vertical += [(coord[0], right)]
        else:
            scene[coord[0], left:right+1] = 2
            open_horizontal += [(coord[0]-1, coord[1])]
        
    
   


In [582]:
# Question 1 (important! ignore y coordinates lower than first #)

In [583]:
np.sum(np.isin(scene[miny:,:], [2,3]))

39367

In [584]:
# Question 2

In [585]:
np.sum(scene[miny:,:] == 2)

33061